In [2]:
import tkinter as tk
from tkinter import ttk
from verbecc import Conjugator
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from tkinter import messagebox


In [19]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\ROYAL
[nltk_data]     COMPUTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ROYAL COMPUTER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\ROYAL
[nltk_data]     COMPUTER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\ROYAL
[nltk_data]     COMPUTER\AppData\Roaming\nltk_data...


True

In [3]:

cg = Conjugator(lang='fr')

In [4]:


class Verbe:
    def __init__(self, verbe):
        self.verbe = verbe

    def instanciation_verbec(self):
        return cg.conjugate(self.verbe)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Conjugaison et Analyse de Phrase")

        self.create_widgets()
    
    def create_widgets(self):
        self.tab_control = ttk.Notebook(self.root)

        self.conjugaison_tab = ttk.Frame(self.tab_control)
        self.analyse_phrase_tab = ttk.Frame(self.tab_control)

        self.tab_control.add(self.conjugaison_tab, text="Conjugaison")
        self.tab_control.add(self.analyse_phrase_tab, text="Analyse de Phrase")
        self.tab_control.pack(expand=1, fill="both")

        self.create_conjugaison_tab()
        self.export_pdf_button = tk.Button(self.conjugaison_tab, text="Exporter PDF", command=self.export_to_pdf)
        self.export_pdf_button.pack()
        self.create_analyse_phrase_tab()
        

    def create_conjugaison_tab(self):
        self.verbe_label = tk.Label(self.conjugaison_tab, text="Saisissez un verbe:")
        self.verbe_label.pack()

        self.verbe_entry = tk.Entry(self.conjugaison_tab)
        self.verbe_entry.pack()

        self.submit_button = tk.Button(self.conjugaison_tab, text="Conjuguer", command=self.conjuguer_verbe)
        self.submit_button.pack()

        self.mode_label = tk.Label(self.conjugaison_tab, text="Choisissez un mode:")
        self.mode_label.pack()

        self.mode_combobox = ttk.Combobox(self.conjugaison_tab, values=["", "indicatif", "imperatif", "conditionnel", "subjonctif", "participe"])
        self.mode_combobox.pack()
        self.mode_combobox.bind("<<ComboboxSelected>>", self.update_temps_checkbuttons)

        self.temps_frame = tk.Frame(self.conjugaison_tab)
        self.temps_frame.pack()

        self.result_text = tk.Text(self.conjugaison_tab)
        self.result_text.pack()

        self.temps_checkbuttons = {}


    def create_analyse_phrase_tab(self):
        self.phrase_label = tk.Label(self.analyse_phrase_tab, text="Saisissez une phrase:")
        self.phrase_label.pack()

        self.phrase_entry = tk.Entry(self.analyse_phrase_tab)
        
        self.phrase_entry.pack()

        self.analyse_button = tk.Button(self.analyse_phrase_tab, text="Analyser", command=self.analyser_phrase)
        self.analyse_button.pack()

        self.analyse_text = tk.Text(self.analyse_phrase_tab)
        self.analyse_text.pack()


    def conjuguer_verbe(self):
        verbe = self.verbe_entry.get()
        self.verbe = Verbe(verbe)
        self.resultat = self.verbe.instanciation_verbec()
        self.afficher_conjugaison_complete()

    def update_temps_checkbuttons(self, event=None):
        for widget in self.temps_frame.winfo_children():
            widget.destroy()

        mode = self.mode_combobox.get()
        self.temps_checkbuttons = {}

        if mode and mode in self.resultat["moods"]:
            for temps in self.resultat["moods"][mode].keys():
                var = tk.BooleanVar()
                checkbutton = tk.Checkbutton(self.temps_frame, text=temps, variable=var, command=self.afficher_conjugaison)
                checkbutton.pack(anchor='w')
                self.temps_checkbuttons[temps] = var

        self.afficher_conjugaison()

    def afficher_conjugaison_complete(self):
        self.result_text.delete(1.0, tk.END)
        for mode in self.resultat["moods"].keys():
            self.result_text.insert(tk.END, f"************Mode {mode.capitalize()}************\n")
            for temps in self.resultat["moods"][mode].keys():
                self.result_text.insert(tk.END, f"Temps {temps}\n")
                for conjugaison in self.resultat["moods"][mode][temps]:
                    self.result_text.insert(tk.END, conjugaison + "\n")
                self.result_text.insert(tk.END, "\n")

    def afficher_conjugaison(self):
        self.result_text.delete(1.0, tk.END)
        mode = self.mode_combobox.get()

        if mode and mode in self.resultat["moods"]:
            self.result_text.insert(tk.END, f"************Mode {mode.capitalize()}************\n")
            for temps, var in self.temps_checkbuttons.items():
                if var.get():
                    self.result_text.insert(tk.END, f"Temps {temps}\n")
                    for conjugaison in self.resultat["moods"][mode][temps]:
                        self.result_text.insert(tk.END, conjugaison + "\n")
                    self.result_text.insert(tk.END, "\n")
            if not any(var.get() for var in self.temps_checkbuttons.values()):
                for temps in self.resultat["moods"][mode].keys():
                    self.result_text.insert(tk.END, f"Temps {temps}\n")
                    for conjugaison in self.resultat["moods"][mode][temps]:
                        self.result_text.insert(tk.END, conjugaison + "\n")
                    self.result_text.insert(tk.END, "\n")
        else:
            self.afficher_conjugaison_complete()


    def analyser_phrase(self):
        phrase = self.phrase_entry.get()
        analyse_result = self.analyse_syntaxique(phrase)
        self.afficher_analyse_phrase(analyse_result)

    def analyse_syntaxique(self, phrase):
        tokens = word_tokenize(phrase)
        tagged = pos_tag(tokens)

        type_phrase = "Déclarative"
        if phrase.endswith('?'):
            type_phrase = "Interrogative"
        elif phrase.endswith('!'):
            type_phrase = "Exclamative"
        elif tagged[0][1] == 'VB':
            type_phrase = "Impérative"

        forme_phrase = "Affirmative"
        if "pas" in tokens or "n'" in tokens:
            forme_phrase = "Négative"

        return {
            "Type": type_phrase,
            "Forme": forme_phrase
            }
    def afficher_analyse_phrase(self, analyse_result):
        self.analyse_text.delete(1.0, tk.END)
        self.analyse_text.insert(tk.END, f"Type de phrase: {analyse_result['Type']}\n")
        self.analyse_text.insert(tk.END, f"Forme de phrase: {analyse_result['Forme']}\n\n")
    def export_to_pdf(self):
        
        analyse_content = self.result_text.get(1.0, tk.END)

        c = canvas.Canvas("conjuguaison.pdf", pagesize=letter)
        c.drawString(100, 750, "Conjuguaison de verbe")
        c.drawString(100, 730, "-----------------------------------------")

        lines_per_page = 40

        lines = analyse_content.split('\n')
        num_lines = len(lines)
        num_pages = num_lines // lines_per_page + (1 if num_lines % lines_per_page != 0 else 0)

        y_start = 700
        current_line = 0
        for page_num in range(num_pages):
            y_end = y_start - (lines_per_page * 15)

            while current_line < num_lines and y_start > y_end:
                line = lines[current_line]
                c.drawString(100, y_start, line)
                y_start -= 15
                current_line += 1
            if current_line < num_lines:
                c.showPage() 
                c.drawString(100, 750, "Conjuguaison de verbe")
                c.drawString(100, 730, "-----------------------------------------")
                y_start = 700

        # Enregistrer le PDF
        c.save()
        messagebox.showinfo("Export PDF", "Conjuguaison exportée en PDF avec succès !")
            
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()
